In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2024-03-08T06:06:03.112784-05:00

Python implementation: CPython
Python version       : 3.7.11
IPython version      : 7.10.1

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 4.15.0-161-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 40
Architecture: 64bit



In [3]:
%watermark --gpu

GPU Info: 
  GPU 0: Tesla V100-DGXS-32GB
  GPU 1: Tesla V100-DGXS-32GB
  GPU 2: Tesla V100-DGXS-32GB
  GPU 3: Tesla V100-DGXS-32GB



In [4]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from dask import dataframe as dd
from dask.delayed import delayed
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

import optuna
import gc
import logging

In [5]:
%watermark --iversions

optuna : 2.10.1
pandas : 1.3.5
numpy  : 1.21.6
logging: 0.5.1.2
dask   : 2022.2.0
xgboost: 1.6.2



In [6]:
!nvidia-smi -L

GPU 0: Tesla V100-DGXS-32GB (UUID: GPU-d752af09-1f62-bf3b-4f70-78b84e9e41f6)
GPU 1: Tesla V100-DGXS-32GB (UUID: GPU-054a4a35-f98a-3ebc-9100-0f697c246b43)
GPU 2: Tesla V100-DGXS-32GB (UUID: GPU-454525d4-bebd-7fb8-0ba3-3b85e2f99abd)
GPU 3: Tesla V100-DGXS-32GB (UUID: GPU-af0fb74e-f5eb-0833-17ff-494cc6bdcee1)


In [7]:
cluster = LocalCUDACluster(n_workers=4)
client = Client(cluster)

In [8]:
!nvidia-smi

Fri Mar  8 06:06:11 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.142.00   Driver Version: 450.142.00   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-DGXS...  On   | 00000000:07:00.0  On |                    0 |
| N/A   39C    P0    51W / 300W |    780MiB / 32505MiB |      7%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-DGXS...  On   | 00000000:08:00.0 Off |                    0 |
| N/A   

In [9]:
def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
    
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)
 
def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

In [10]:
train_folds = []
val_folds = []
train_ys = []
val_ys = []

for i in range(5):
    print(f'Loading fold {i}')
    train_fold_d = delayed(pd.read_csv)(f'../input/xgtrain_fold_{i}_drop_shap_3_p2fe.csv.gz')
    train_fold = dd.from_delayed(train_fold_d)
    
    val_fold_d = delayed(pd.read_csv)(f'../input/xgval_fold_{i}_drop_shap_3_p2fe.csv.gz')
    val_fold = dd.from_delayed(val_fold_d)
    
    
    train_y = train_fold['target']
    train_fold = train_fold[train_fold.columns.difference(['target'])]
    
    val_y = val_fold['target']
    val_fold = val_fold[val_fold.columns.difference(['target'])]
    
    train_folds.append(train_fold)
    val_folds.append(val_fold)
    
    train_ys.append(train_y)
    val_ys.append(val_y)

Loading fold 0
Loading fold 1
Loading fold 2
Loading fold 3
Loading fold 4


In [11]:
train = pd.read_csv('../input/train.csv.zip')

target = train['target'].values
target

array([0, 0, 0, ..., 0, 0, 0])

In [12]:
train_oof = np.zeros((target.shape[0],))

num_round = 1000

def objective(trial):
        
    params = {
        'objective': trial.suggest_categorical('objective',['binary:logistic']),
        'tree_method': trial.suggest_categorical('tree_method',['gpu_hist']),  # 'gpu_hist','hist'
        'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
        'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3,1.0),
        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001,0.1),
        #'n_estimators': trial.suggest_categorical('n_estimators', [1000]),
        'max_depth': trial.suggest_int('max_depth', 3, 25),
        #'random_state': trial.suggest_categorical('random_state', [24,48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1,300),
        'eval_metric': trial.suggest_categorical('eval_metric',['logloss']),

    }

    kf = KFold(5, shuffle=True, random_state=137)

    for i, (train_index, val_index) in enumerate(kf.split(train,target)):
        dtrain = xgb.dask.DaskDMatrix(client, train_folds[i].values, train_ys[i], enable_categorical=True)
        dval = xgb.dask.DaskDMatrix(client, val_folds[i].values, val_ys[i], enable_categorical=True)
        
        output = xgb.dask.train(client, params, dtrain, num_round)
        booster = output['booster']  # booster is the trained model
        booster.set_param({'predictor': 'gpu_predictor'})
        predictions = xgb.dask.predict(client, booster, dval)
        predictions = predictions.compute()
        train_oof[val_index] = predictions
        del dtrain, dval, output
        gc.collect()
        gc.collect()

    gini = gini_normalized(target, train_oof)
    
    return gini

In [13]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)  # Setup the root logger.
logger.addHandler(logging.FileHandler("optuna_xgb_output_9.log", mode="w"))

optuna.logging.enable_propagation()  # Propagate logs to the root logger.
optuna.logging.disable_default_handler()  # Stop showing logs in sys.stderr.

study = optuna.create_study(storage="sqlite:///xgb_optuna_porto_9.db", study_name="five_fold_optuna_xgb_9", direction='maximize')

In [14]:
%%time
logger.info("Start optimization.")
study.optimize(objective, n_trials=3)

/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
distributed.utils_perf - WARNING - full garbage collections took 32% CPU time recently (threshold: 10%)


CPU times: user 23.2 s, sys: 3.94 s, total: 27.2 s
Wall time: 3min 26s


In [15]:
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.head()

,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_objective,params_subsample,params_tree_method,state
0,0,0.275419,0.005911,0.498093,logloss,5.980028,0.004001,18,82,binary:logistic,0.836637,gpu_hist,COMPLETE
1,1,0.278095,0.042712,0.333441,logloss,0.694855,0.007536,20,223,binary:logistic,0.535148,gpu_hist,COMPLETE
2,2,0.276776,0.001489,0.533250,logloss,0.868450,0.040532,7,246,binary:logistic,0.971326,gpu_hist,COMPLETE


In [16]:
%%time
study.optimize(objective, n_trials=5)
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.to_csv('optuna_xgb_output_9.csv', index=False)
df.head(8)

/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
distributed.utils_perf - WARNING - full garbage collections took 68% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 68% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 68% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 68% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 68% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 69% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 69% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 69% CPU time recently (thre

CPU times: user 48.3 s, sys: 8.92 s, total: 57.2 s
Wall time: 7min 39s


,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_objective,params_subsample,params_tree_method,state
0,0,0.275419,0.005911,0.498093,logloss,5.980028,0.004001,18,82,binary:logistic,0.836637,gpu_hist,COMPLETE
1,1,0.278095,0.042712,0.333441,logloss,0.694855,0.007536,20,223,binary:logistic,0.535148,gpu_hist,COMPLETE
2,2,0.276776,0.001489,0.533250,logloss,0.868450,0.040532,7,246,binary:logistic,0.971326,gpu_hist,COMPLETE
3,3,0.259737,1.484722,0.566795,logloss,0.147925,0.001349,11,178,binary:logistic,0.698475,gpu_hist,COMPLETE
4,4,0.241890,0.047991,0.670709,logloss,0.085437,0.057818,16,137,binary:logistic,0.576291,gpu_hist,COMPLETE
5,5,0.275030,0.004847,0.467533,logloss,1.311431,0.004526,16,244,binary:logistic,0.954295,gpu_hist,COMPLETE
6,6,0.279325,0.004213,0.775214,logloss,0.002227,0.007283,22,280,binary:logistic,0.562885,gpu_hist,COMPLETE
7,7,0.244783,0.004889,0.614848,logloss,0.008510,0.060487,13,122,binary:logistic,0.418770,gpu_hist,COMPLETE


In [17]:
study.best_trial.params

{'alpha': 0.004212656482907152,
 'colsample_bytree': 0.7752139823843462,
 'eval_metric': 'logloss',
 'lambda': 0.0022265595625634814,
 'learning_rate': 0.007282990864957459,
 'max_depth': 22,
 'min_child_weight': 280,
 'objective': 'binary:logistic',
 'subsample': 0.5628845656749031,
 'tree_method': 'gpu_hist'}

In [18]:
df.value.max()

0.2793245115212991

In [19]:
%%time
study.optimize(objective, n_trials=100)
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.to_csv('optuna_xgb_output_9.csv', index=False)
df.head(20)

/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
distributed.utils_perf - WARNING - full garbage collections took 73% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 73% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 73% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 74% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 74% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 74% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 74% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 74% CPU time recently (thre

CPU times: user 1h 3min 54s, sys: 13min 13s, total: 1h 17min 7s
Wall time: 12h 35min 22s


,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_objective,params_subsample,params_tree_method,state
0,0,0.275419,0.005911,0.498093,logloss,5.980028,0.004001,18,82,binary:logistic,0.836637,gpu_hist,COMPLETE
1,1,0.278095,0.042712,0.333441,logloss,0.694855,0.007536,20,223,binary:logistic,0.535148,gpu_hist,COMPLETE
2,2,0.276776,0.001489,0.533250,logloss,0.868450,0.040532,7,246,binary:logistic,0.971326,gpu_hist,COMPLETE
3,3,0.259737,1.484722,0.566795,logloss,0.147925,0.001349,11,178,binary:logistic,0.698475,gpu_hist,COMPLETE
4,4,0.241890,0.047991,0.670709,logloss,0.085437,0.057818,16,137,binary:logistic,0.576291,gpu_hist,COMPLETE
5,5,0.275030,0.004847,0.467533,logloss,1.311431,0.004526,16,244,binary:logistic,0.954295,gpu_hist,COMPLETE
6,6,0.279325,0.004213,0.775214,logloss,0.002227,0.007283,22,280,binary:logistic,0.562885,gpu_hist,COMPLETE
7,7,0.244783,0.004889,0.614848,logloss,0.008510,0.060487,13,122,binary:logistic,0.418770,gpu_hist,COMPLETE
8,8,0.276010,0.004924,0.434298,logloss,0.770845,0.043544,5,60,binary:logistic,0.865351,gpu_hist,COMPLETE
9,9,0.278145,0.303686,0.632071,logloss,0.015036,0.010512,4,177,binary:logistic,0.503739,gpu_hist,COMPLETE


In [20]:
df.value.max()

0.28364434323466076

In [21]:
study.best_trial.params

{'alpha': 0.005526575111708585,
 'colsample_bytree': 0.47370401892621367,
 'eval_metric': 'logloss',
 'lambda': 1.0960444593037473,
 'learning_rate': 0.015108466251527685,
 'max_depth': 24,
 'min_child_weight': 279,
 'objective': 'binary:logistic',
 'subsample': 0.9146800676408706,
 'tree_method': 'gpu_hist'}